In [31]:
import pandas as pd
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from datetime import datetime

In [32]:
spark = SparkSession.builder.appName("chicago_crimes").getOrCreate()

In [33]:
crimes = spark.read.csv("gs://chicago_crimes_bucket/Crimes_-_2001_to_Present.csv",inferSchema=True,header = True)

In [4]:
crimes = crimes.cache()

In [5]:
crimes.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Historical Wards 2003-2015: integer (nullable = true)
 |-- Zip Codes: integer (nullable = tr

In [6]:
cols = ("Community Areas","Historical Wards 2003-2015","Zip Codes","Census Tracts","Wards","Boundaries - ZIP Codes","Police Districts","Police Beats")

crimes = crimes.drop(*cols)

In [7]:
crimes.count()

7673480

In [8]:
len(crimes.columns)

22

In [9]:
crimes=crimes.na.drop("any")

In [10]:
crimes.count()

6978932

In [36]:
crimes.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Historical Wards 2003-2015: integer (nullable = true)
 |-- Zip Codes: integer (nullable = tr

In [11]:
convert_dt =  udf(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'), TimestampType())

crimes = crimes.withColumn('Date_time', convert_dt(col('Date'))).drop("Date")

In [12]:
crimes = crimes.withColumn('Updated_date', convert_dt(col('Updated On'))).drop("Updated On")

In [14]:
crimes.select(crimes["Date_time"]).show(5)

+-------------------+
|          Date_time|
+-------------------+
|2015-09-05 13:30:00|
|2015-09-04 11:30:00|
|2015-09-05 12:45:00|
|2015-09-05 13:00:00|
|2015-09-05 10:55:00|
+-------------------+
only showing top 5 rows



In [29]:
crimes.show(5)

+--------+-----------+-------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+------------+-------------+--------------------+-------------------+-------------------+
|      ID|Case Number|              Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|    Latitude|    Longitude|            Location|          Date_time|       Updated_date|
+--------+-----------+-------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+------------+-------------+--------------------+-------------------+-------------------+
|10224738|   HY411648|    043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true| 924|       9|  12|            61|     

In [35]:
crimes.select("Primary Type").distinct().show(n = 20)

+--------------------+
|        Primary Type|
+--------------------+
|OFFENSE INVOLVING...|
|CRIMINAL SEXUAL A...|
|            STALKING|
|PUBLIC PEACE VIOL...|
|           OBSCENITY|
|NON-CRIMINAL (SUB...|
|               ARSON|
|   DOMESTIC VIOLENCE|
|            GAMBLING|
|   CRIMINAL TRESPASS|
|             ASSAULT|
|      NON - CRIMINAL|
|LIQUOR LAW VIOLATION|
| MOTOR VEHICLE THEFT|
|               THEFT|
|             BATTERY|
|             ROBBERY|
|            HOMICIDE|
|           RITUALISM|
|    PUBLIC INDECENCY|
+--------------------+
only showing top 20 rows



In [27]:
crimes.write.option("header",True).csv("gs://chicago_crimes_bucket/crimes_cleaned_dataset.csv")